Ver: https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

### Se debe tener las últimas versiones de Keras y TensorFlow!

In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
shuffle_data = True
dog_breed_train_path = 'Dog Breed/train/'
dog_breed_val_path = 'Dog Breed/val/'
dog_breed_labels_path = 'Dog Breed/labels.csv'

# read addresses and labels
labels = pd.read_csv(dog_breed_labels_path)

# to shuffle data
if shuffle_data:
    labels = labels.sample(frac=1)

### Ojo: el siguiente bloque de código sólo se ejecuta una vez; sirve para mover los archivos de imágenes de cada clase a sus respectivas carpetas

In [3]:
# load dataset and labels into variables

# use 20% of the train set to create a validation set and another 20% for a test set
train_labels = labels.iloc[:int(0.8*len(labels))]
val_labels = labels.iloc[int(0.8*len(labels)):]

# a numpy array to save the mean of the images

# loop over train addresses
for i, (index, img_id, label) in enumerate(train_labels.itertuples()):
    # print how many images are saved every 1000 images
    if (i+1) % 1000 == 0 or i+1 == len(train_labels):
        print ('Train data: {0}/{1}'.format(i+1, len(train_labels)))
        print('Processing: original_index={0}, id={1}, label={2}'.format(index, img_id, label))

    # move the image to a subdirectory named after its label
    if not os.path.isdir(os.path.join(dog_breed_train_path, label)):
        os.mkdir(os.path.join(dog_breed_train_path, label))
    os.rename(dog_breed_train_path+img_id+'.jpg', dog_breed_train_path+label+'/'+img_id+'.jpg')

for i, (index, img_id, label) in enumerate(val_labels.itertuples()):
    # print how many images are saved every 1000 images
    if (i+1) % 1000 == 0 or i+1 == len(val_labels):
        print ('Validation data: {0}/{1}'.format(i+1, len(val_labels)))
        print('Processing: original_index={0}, id={1}, label={2}'.format(index, img_id, label))

    # move the image to a subdirectory named after its label
    if not os.path.isdir(os.path.join(dog_breed_val_path, label)):
        os.mkdir(os.path.join(dog_breed_val_path, label))
    os.rename(dog_breed_train_path+img_id+'.jpg', dog_breed_val_path+label+'/'+img_id+'.jpg')

# aqui va el centrado de datos

Train data: 1000/8177
Processing: original_index=8540, id=d66ec4c83a620cca6ebf05ab9d162fcd, label=cairn
Train data: 2000/8177
Processing: original_index=9036, id=e316925eb1cf7cdeb1ffaab7424e231d, label=shetland_sheepdog
Train data: 3000/8177
Processing: original_index=3264, id=511bfe35ff282294f6129c55bd6c33f6, label=malinois
Train data: 4000/8177
Processing: original_index=3337, id=52e7f48e18a9a55da9846d56821d5f69, label=cardigan
Train data: 5000/8177
Processing: original_index=8096, id=ca39d409bccf8034d2d11b7b44d8e0a1, label=vizsla
Train data: 6000/8177
Processing: original_index=4688, id=7589719a4f8dddeb1562fd1bff7f714e, label=schipperke
Train data: 7000/8177
Processing: original_index=8922, id=dfe45c3b288b8224eb17dbdaf1706496, label=irish_terrier
Train data: 8000/8177
Processing: original_index=9967, id=fa3bc3e096a2967f26113992b29b23b5, label=toy_terrier
Train data: 8177/8177
Processing: original_index=2584, id=403dc79fe2e7b3128ab675ef6762754a, label=labrador_retriever
Validation da

### Hay que jugar un poco con la arquitectura de la CNN. Mi PC no aguanta un modelo que tenga más de ~12 millones de parámetros, así que las opciones son achicar las imágenes de entrada (pic_dimension) y/o disminuir la cantidad de neuronas del modelo:

In [131]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

n_classes = 120
pic_dimension = 112

modelo_1 = Sequential()
modelo_1.add(Conv2D(32, (3, 3), padding='same', input_shape=(pic_dimension, pic_dimension, 3)))
modelo_1.add(Activation('relu'))
modelo_1.add(MaxPooling2D(pool_size=(2, 2)))
#modelo_1.add(Dropout(0.3))
modelo_1.add(Conv2D(64, (3, 3), padding='same'))
modelo_1.add(Activation('relu'))
modelo_1.add(MaxPooling2D(pool_size=(2, 2)))
#modelo_1.add(Dropout(0.3))
modelo_1.add(Conv2D(128, (3, 3), padding='same'))
modelo_1.add(Activation('relu'))
modelo_1.add(MaxPooling2D(pool_size=(2, 2)))
#modelo_1.add(Dropout(0.3))
modelo_1.add(Flatten())
modelo_1.add(Dense(456))
modelo_1.add(Activation('relu'))
modelo_1.add(Dropout(0.5))
modelo_1.add(Dense(n_classes))
modelo_1.add(Activation('softmax'))
modelo_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_214 (Conv2D)          (None, 112, 112, 32)      896       
_________________________________________________________________
activation_298 (Activation)  (None, 112, 112, 32)      0         
_________________________________________________________________
max_pooling2d_121 (MaxPoolin (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_215 (Conv2D)          (None, 56, 56, 64)        18496     
_________________________________________________________________
activation_299 (Activation)  (None, 56, 56, 64)        0         
_________________________________________________________________
max_pooling2d_122 (MaxPoolin (None, 28, 28, 64)        0         
_________________________________________________________________
conv2d_216 (Conv2D)          (None, 28, 28, 128)       73856     
__________

In [132]:
# compilación del modelo
from keras.callbacks import Callback
from keras.optimizers import SGD, RMSprop

modelo_1.compile(optimizer=RMSprop(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

La cantidad de datos del dataset es muy poco (10222 imagenes) respecto a la dimensionalidad del mismo. Por lo tanto, es útil aumentar artificialmente el dataset aplicando distorsiones aleatorias a cada imagen, de manera que el modelo en entrenamiento nunca vea la misma imagen más de una vez.

In [133]:
from keras.preprocessing.image import ImageDataGenerator

batch_size = 64

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rotation_range=40,
        rescale=1./255,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

# this is the augmentation configuration we will use for validation:
# only rescaling
val_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        dog_breed_train_path,  # this is the target directory
        target_size=(pic_dimension, pic_dimension),  # all images will be resized to 112x112
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = val_datagen.flow_from_directory(
        dog_breed_val_path,
        target_size=(pic_dimension, pic_dimension),
        batch_size=batch_size,
        class_mode='categorical')

Found 8177 images belonging to 120 classes.
Found 2045 images belonging to 120 classes.


In [134]:
# entrenamiento de la CNN.
import tensorflow as tf

conf = tf.ConfigProto()
conf.gpu_options.allow_growth = True
sess = tf.Session(config=conf)

from keras import backend as k
k.set_session(sess)

results_test = {'test_loss': [], 'test_acc': []}

results = modelo_1.fit_generator(
        train_generator, 
        steps_per_epoch=10222//batch_size, # tamaño del dataset completo//tamaño del batch 
        epochs=50,
        validation_data=validation_generator,
        validation_steps=2045//batch_size,
        #callbacks=[lrate]
        )
modelo_1.save_weights('modelo_1.h5')

Epoch 1/50
159/159 [==============================] - 65s 410ms/step - loss: 4.8881 - acc: 0.0101 - val_loss: 4.7579 - val_acc: 0.0166
Epoch 2/50
159/159 [==============================] - 56s 354ms/step - loss: 4.6953 - acc: 0.0224 - val_loss: 4.5743 - val_acc: 0.0277
Epoch 3/50
159/159 [==============================] - 57s 360ms/step - loss: 4.5506 - acc: 0.0296 - val_loss: 4.4368 - val_acc: 0.0393
Epoch 4/50
159/159 [==============================] - 57s 358ms/step - loss: 4.4329 - acc: 0.0408 - val_loss: 4.3152 - val_acc: 0.0585
Epoch 5/50
159/159 [==============================] - 60s 375ms/step - loss: 4.3497 - acc: 0.0495 - val_loss: 4.2232 - val_acc: 0.0691
Epoch 6/50
159/159 [==============================] - 56s 351ms/step - loss: 4.2607 - acc: 0.0645 - val_loss: 4.1381 - val_acc: 0.0746
Epoch 7/50
159/159 [==============================] - 56s 353ms/step - loss: 4.1797 - acc: 0.0746 - val_loss: 4.1579 - val_acc: 0.0771
Epoch 8/50
159/159 [==============================] - 5